# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [2]:
# from pymongo import MongoClient
# client = MongoClient("")
from pymongo import MongoClient
import pandas as pd 
import time 
client = MongoClient("localhost:27017")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [15]:
# Your Code
client
db = client["Ironhack"]
c = db.get_collection("Companies")

In [16]:
result = list(c.find({'name': 'Babelgum'}, {'_id':0,'name':1}))
result

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [17]:
# Your Code
filter_ = {'number_of_employees':{'$gt':5000}}
projection =  {"_id":0, "name":1, "number_of_employees":1}

result = list(c.find(filter_, projection).sort('number_of_employees',-1).limit(20))
result

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [18]:
# Your Code
condition1 = {'founded_year':{'$gte':2000}}
condition2 = {'founded_year':{'$lte':2005}}

#filter_ = {'$and':[condition1],[condition2]}
projection =  {"_id":0, "name":1, "founded_year":1}
           
result = list(c.find({'$and':[condition1,condition2]}, projection))
result

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'foun

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [19]:
# Your Code
condition1 = {'milestones.ipo':{'$gt':'100000000'}}
condition2 = {'founded_year':{'$lt':2010}}

projection =  {"_id":0, "name":1, "milestones.ipo":1}
result = list(c.find({'$and':[condition1,condition2]}, projection))
result

[]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [20]:
# Your Code
condition1 = {'number_of_employees':{'$lt':1000}}
condition2 = {'founded_year':{'$lt':2005}}
projection =  {"_id":0, "name":1}

result = list(c.find({'$and':[condition1,condition2]}, projection).sort('number_of_employees',-1).limit(10))
result

[{'name': 'Infinera Corporation'},
 {'name': 'NorthPoint Communications Group'},
 {'name': '888 Holdings'},
 {'name': 'Forrester Research'},
 {'name': 'SonicWALL'},
 {'name': 'Webmetrics'},
 {'name': 'Cornerstone OnDemand'},
 {'name': 'Mozilla'},
 {'name': 'Buongiorno'},
 {'name': 'Yelp'}]

### 6. All the companies that don't include the `partners` field.

In [48]:
# Your Code
filter_ = {'partners':{'$exists': False }}

result = list(c.find(filter_,{}))
result

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [47]:
# Your Code
filter_ = {'category_code': 'null'}
projection =  {"_id":0, "name":1}

result = list(c.find(filter_, projection))
result

[]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [23]:
# Your Code
condition1 = {'number_of_employees':{'$gte':100}}
condition2 = {'number_of_employees':{'$lt':1000}}

projection =  {"_id":0, "name":1, "number_of_employees":1}
result = list(c.find({'$and':[condition1,condition2]}, projection))
result

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [42]:
# Your Code
projection= {"_id":0, "name":1}
result = list(c.find({},projection).sort("milestones.ipo",-1))
result

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Twitter'},
 {'name': 'StumbleUpon'},
 {'name': 'Gizmoz'},
 {'name': 'Scribd'},
 {'name': 'Slacker'},
 {'name': 'Lala'},
 {'name': 'Helio'},
 {'name': 'eBay'},
 {'name': 'MeetMoi'},
 {'name': 'Joost'},
 {'name': 'CBS'},
 {'name': 'Viacom'},
 {'name': 'Babelgum'},
 {'name': 'Plaxo'},
 {'name': 'Cisco'},
 {'name': 'Yahoo!'},
 {'name': 'Powerset'},
 {'name': 'Technorati'},
 {'name': 'SpinVox'},
 {'name': 'AddThis'},
 {'name': 'OpenX'},
 {'name': 'Mahalo'},
 {'name': 'Sparter'},
 {'name': 'Kyte'},
 {'name': 'Veoh'},
 {'name': 'Gannett'},
 {'name': 'Thoof'},
 {'name': 'Jingle Networks'},
 {'name': 'Info'},
 {'name': 'JotSpot'},
 {'name': 'Meetup'},
 {'name': 'Mercora'},
 {'name': 'NetRatings'},
 {'name': 'LifeLock'},
 {'name': 'Wesabe'},
 {'name': 'Jangl

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [38]:
# Your Code
result = list(c.find().sort('number_of_employees',-1).limit(10))
result

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'permalink': 'siemens',
  'crunchbase_url': 'http://www.crunchbase.com/company/siemens',
  'homepage_url': 'http://www.siemens.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'Siemens',
  'category_code': 'hardware',
  'number_of_employees': 405000,
  'founded_year': 1847,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'automation, building-technologies, drive-technology, energy',
  'alias_list': '',
  'email_address': 'contact@siemens.com',
  'phone_number': '49 89 636 34134',
  'description': 'Electronics and Electrical Engineering',
  'created_at': 'Thu Jul 31 09:29:43 UTC 2008',
  'updated_at': 'Thu Nov 28 20:32:55 UTC 2013',
  'overview': '<p>Siemens AG, an electronics and electrical engineering company, operates in the industry, energy, and healthcare sectors worldwide. 

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [25]:
# Your Code
condition1 = {'founded_month':{'$gte':7}}
condition2 = {'founded_month':{'$lte':12}}

projection =  {"_id":0, "name":1}
result = list(c.find({'$and':[condition1,condition2]}, projection).limit(1000))
result

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'eBay'},
 {'name': 'Joost'},
 {'name': 'Plaxo'},
 {'name': 'Powerset'},
 {'name': 'Kyte'},
 {'name': 'Thoof'},
 {'name': 'Jingle Networks'},
 {'name': 'LifeLock'},
 {'name': 'Wesabe'},
 {'name': 'SmugMug'},
 {'name': 'Google'},
 {'name': 'Skype'},
 {'name': 'Pando Networks'},
 {'name': 'Ikan'},
 {'name': 'delicious'},
 {'name': 'AllPeers'},
 {'name': 'Wize'},
 {'name': 'SellABand'},
 {'name': 'iContact'},
 {'name': 'MeeVee'},
 {'name': 'blinkx'},
 {'name': 'Zlio'},
 {'name': 'Yelp'},
 {'name': 'Jaiku'},
 {'name': 'Yapta'},
 {'name': 'Fleck'},
 {'name': 'SideStep'},
 {'name': 'RockYou'},
 {'name': 'Instructables'},
 {'name': 'Netvibes'},
 {'name': 'Swivel'},
 {'name': 'Slide'},
 {'name': 'CrowdVine'},
 {'name': 'TheFind'},
 {'name': 'Sway'},
 {'name': 'CornerWorld'},
 {'name': 'FlickIM'},
 {'name': 'Terabitz'},
 {'name': 'Kaboodle'},
 {'name': 'GigaOM'},
 {'name': 'Redfin'},
 {'name': 'Wink'},

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [26]:
# Your Code
condition1 = {'founded_year':{'$lt':2000}}
condition2 = {'acquisition.price_amount':{'$gt':10000000}}

projection =  {"_id":0, "name":1}
result = list(c.find({'$and':[condition1,condition2]}, projection))
result

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'},
 {'name': 'Neopets'},
 {'name': 'Sun Microsystems'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Sabre'},
 {'name': 'Lastminute'},
 {'name': 'Sportsline'},
 {'name': 'Digitas'},
 {'name': 'Webshots'},
 {'name': 'MIVA'},
 {'name': 'iVillage'},
 {'name': 'PriceGrabber'},
 {'name': 'LinkShare'},
 {'name': 'Kaboose'},
 {'name': 'Fastclick'},
 {'name': 'Eloqua'},
 {'name': 'Omniture'},
 {'name': 'Interwoven'},
 {'name': 'HowStuffWorks'},
 {'name': 'RealAge'},
 {'name': 'Match'},
 {'name': 'Xdrive'},
 {'name': 'Alexa'},
 {'name': 'Offermatica'},
 {'name': 'ioko'},
 {'name': 'MyNewPlace'},
 {'name': 'DoubleClick'},
 {'name': 'Danger'},
 {'name': 'Direct Hit'},
 {'name': 'BEA Systems'},
 {'name': 'Napster'},
 {'name': 'SafeNet'},
 {'name': 'LowerMyBills'},
 {'name': 'Various'},
 {'name': 'aQuantive'},
 {'name': 'PeopleSupport'},
 {'name': 'Wedding Channel'},
 {'name': 'iC

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [27]:
# Your Code
filter_ = {'acquisition.acquired_year':{'$gt':2010}}

projection =  {"_id":0, "name":1, "acquisition":1}
result = list(c.find(filter_, projection).sort('acquisition.price_amount',-1))
result

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [28]:
# Your Code
result = list(c.find({},{"_id":0, "name":1, "founded_year":1}).sort('founded_year',1))
result

[{'name': 'Flektor', 'founded_year': None},
 {'name': 'Lala', 'founded_year': None},
 {'name': 'SpinVox', 'founded_year': None},
 {'name': 'Gannett', 'founded_year': None},
 {'name': 'Info', 'founded_year': None},
 {'name': 'JotSpot', 'founded_year': None},
 {'name': 'Mercora', 'founded_year': None},
 {'name': 'NetRatings', 'founded_year': None},
 {'name': 'There', 'founded_year': None},
 {'name': 'Tun3r', 'founded_year': None},
 {'name': 'PlaceBlogger', 'founded_year': None},
 {'name': 'Cuts', 'founded_year': None},
 {'name': 'Edgeio', 'founded_year': None},
 {'name': 'revlayer', 'founded_year': None},
 {'name': 'Parakey', 'founded_year': None},
 {'name': 'PowerReviews', 'founded_year': None},
 {'name': 'Going', 'founded_year': None},
 {'name': 'WikiYou', 'founded_year': None},
 {'name': 'DZOIC', 'founded_year': None},
 {'name': 'CollegeWikis', 'founded_year': None},
 {'name': 'ProfileBuilder', 'founded_year': None},
 {'name': 'Truemors', 'founded_year': None},
 {'name': 'Hotswap', 'f

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [29]:
# Your Code
filter_ = {'founded_day':{'$lte':7}}

projection =  {"_id":0, "name":1}
result = list(c.find(filter_, projection).sort('acquisition.price_amount',-1).limit(10))
result

[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'},
 {'name': 'Danger'},
 {'name': 'Clearwell Systems'},
 {'name': 'PrimeSense'},
 {'name': 'Amobee'},
 {'name': 'BlueLithium'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [30]:
# Your Code
condition1 = {'category_code':'web'}
condition2 = {'number_of_employees':{'$gt':4000}}

projection =  {"_id":0, "name":1}
result = list(c.find({'$and':[condition1,condition2]},projection).sort('number_of_employees',1))
result

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'},
 {'name': 'Groupon'},
 {'name': 'Yahoo!'},
 {'name': 'eBay'},
 {'name': 'Experian'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [31]:
# Your Code
condition1 = {'acquisition.price_amount':{'$gt':10000000}}
condition2 = {'acquisition.price_currency_code':'EUR'}

projection =  {"_id":0, "name":1}
result = list(c.find({'$and':[condition1,condition2]},projection))
result

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [32]:
# Your Code
condition1 = {'acquisition.acquired_month':{'$gte':1}}
condition2 = {'acquisition.acquired_month':{'$lte':3}}

projection =  {"_id":0, "name":1, "acquisition":1}
result = list(c.find({'$and':[condition1,condition2]},projection))
result

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [33]:
# Your Code
condition1 = {'founded_year':{'$gte':2000}}
condition2 = {'founded_year':{'$lte':2010}}
condition3 = {'acquisition.acquired_year':{'$gte':2011}}

projection =  {"_id":0, "name":1}
result = list(c.find({'$and':[condition1,condition2,condition3]},projection))
result 

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'Kyte'},
 {'name': 'Jingle Networks'},
 {'name': 'blogTV'},
 {'name': 'delicious'},
 {'name': 'Revision3'},
 {'name': 'iContact'},
 {'name': 'Mashery'},
 {'name': 'Dailymotion'},
 {'name': 'KickApps'},
 {'name': 'Instructables'},
 {'name': 'Netvibes'},
 {'name': 'Dogster'},
 {'name': 'adBrite'},
 {'name': 'Meebo'},
 {'name': 'Seesmic'},
 {'name': 'Zvents'},
 {'name': 'Oodle'},
 {'name': 'Flixster'},
 {'name': 'IceRocket'},
 {'name': 'hi5'},
 {'name': 'Indeed'},
 {'name': 'Grockit'},
 {'name': 'Zenfolio'},
 {'name': 'Wishpot'},
 {'name': 'Retrevo'},
 {'name': 'SelectMinds'},
 {'name': 'Flock'},
 {'name': 'TimeBridge'},
 {'name': 'TokBox'},
 {'name': 'Six Apart'},
 {'name': 'MySpace'},
 {'name': 'Doostang'},
 {'name': 'Docstoc'},
 {'name': 'Vlingo'},
 {'name': 'sezmi'},
 {'name': 'Greystripe'},
 {'name': 'LocaModa'},
 {'name': 'Thumbplay'},
 {'name': 'OMGPOP'},
 {'name': 'Daylife'},
 {'name': 'Behance'},
 {'name': 'At

### 20. All the companies that have been 'deadpooled' after the third year.

In [34]:
# Your Code
filter_ = {'deadpooled_year':{'$gt':3}}
projection =  {"_id":0, "name":1}

result = list(c.find(filter_,projection))
result

[{'name': 'Omnidrive'},
 {'name': 'Babelgum'},
 {'name': 'Sparter'},
 {'name': 'Thoof'},
 {'name': 'Mercora'},
 {'name': 'Wesabe'},
 {'name': 'Jangl SMS'},
 {'name': 'Stickam'},
 {'name': 'AllPeers'},
 {'name': 'EQO'},
 {'name': 'AllofMP3'},
 {'name': 'SellABand'},
 {'name': 'Zlio'},
 {'name': 'Jaiku'},
 {'name': 'Spotplex'},
 {'name': 'TripUp'},
 {'name': 'Fleck'},
 {'name': 'Coghead'},
 {'name': 'Zooomr'},
 {'name': 'Swivel'},
 {'name': 'Slide'},
 {'name': 'PlaceBlogger'},
 {'name': 'TripHub'},
 {'name': 'Sway'},
 {'name': 'adBrite'},
 {'name': 'Edgeio'},
 {'name': 'coComment'},
 {'name': 'Hotelicopter'},
 {'name': 'Ujogo'},
 {'name': 'PeerMe'},
 {'name': 'CollegeWikis'},
 {'name': 'Hotswap'},
 {'name': 'SplashCast'},
 {'name': 'Tangler'},
 {'name': 'TalkPlus'},
 {'name': 'Citizenbay'},
 {'name': 'PermissionTV'},
 {'name': 'WebbAlert'},
 {'name': 'Nirvanix'},
 {'name': 'Meetro'},
 {'name': 'MyToons'},
 {'name': 'KnockaTV'},
 {'name': 'Millsberry'},
 {'name': 'Revver'},
 {'name': 'Yoo

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```